In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline

https://automating-gis-processes.github.io/2017/lessons/L5/interactive-map-bokeh.html

https://www.danielforsyth.me/mapping-nyc-taxi-data/

